In [1]:
import ssl
import os
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
os.environ["REQUESTS_CA_BUNDLE"] = str(r"C:\Users\AL03238\OneDrive - Elevance Health\Documents\Work\Project Intelliauth\license\root.pem")
os.environ["SSL_CERT_FILE"] = str(r"C:\Users\AL03238\OneDrive - Elevance Health\Documents\Work\Project Intelliauth\license\root.pem")

In [2]:
!python --version

Python 3.12.4


In [3]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [4]:
import time

index_name = "medical-chatbot2"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [5]:
index.describe_index_stats()


{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [6]:
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [7]:
# Extract the data from pdf
def load_pdf(data):
    loader =DirectoryLoader(data, glob="*.pdf", loader_cls=PyMuPDFLoader)

    documents = loader.load()
    return documents

In [8]:
extracted_data = load_pdf("C:/Users/AL03238/OneDrive - Elevance Health/Documents/Learnings/GitHub/End-to-End-Medical-Chatbot-using-LLM/data/")


In [9]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=10)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print(len(text_chunks))

9659


In [11]:
index.describe_index_stats()


{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [12]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")

C:\ProgramData\Anaconda3\envs\genAI_v2\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
def query(text):
    query_embedding = embeddings.embed_query(text)
    print(query_embedding)
    query_result = index.query(query_embedding, top_k=3, top_n=2)
    return query_result

In [14]:
# import multiprocessing


In [15]:
# def upsert(t):
#     # Perform operations on the item
#     # Return the result
#     index.upsert(vectors=[{"values": embeddings.embed_query(t.page_content), "metadata": {"text": t.page_content}}])
#     pass


In [16]:
# num_processors = multiprocessing.cpu_count()
# pool = multiprocessing.Pool(processes=num_processors)


In [17]:
# pool.map(upsert, text_chunks)
# pool.close()
# pool.join()


In [18]:
from tqdm import tqdm
for i, t in enumerate(tqdm(text_chunks)):

    # Upsert each word embedding into the index
    index.upsert(vectors=[{"id": str(i), "values": embeddings.embed_query(t.page_content), "metadata": {"text": t.page_content}}])

100%|██████████| 9659/9659 [51:26<00:00,  3.13it/s]  


In [19]:
from langchain.llms import CTransformers
llm = CTransformers(model=r"C:\Users\AL03238\OneDrive - Elevance Health\Documents\Learnings\GitHub\End-to-End-Medical-Chatbot-using-LLM\model\llama-2-13b-chat.ggmlv3.q5_1.bin",
                    model_type = "llama",
                    config={"max_new_tokens":1024,
                            'temperature':0.2})

In [20]:
from langchain_pinecone import PineconeVectorStore

vectorstore = PineconeVectorStore(index=index, embedding=embeddings)

In [21]:
from langchain import PromptTemplate


In [22]:
prompt_template='''
Use the following pieces of information to answer the question:
If you don't know the answer to the question, just say you dont know, dont try to make up an answer to the question.

Context: {context}
Question: {question}
Only return the helpful answer to the question and nothing else.
Helpful answer: 
'''

In [23]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"], output_variables=["helpful_answer"])


In [24]:
# RetrievalQA
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vectorstore.as_retriever(), chain_type_kwargs={"prompt": PROMPT}
)

In [28]:
s = time.time()
question = "What are Antigen?"
result = qa_chain({"query": question})
result["result"]

TypeError: 'module' object is not callable. Did you mean: 'time.time(...)'?

In [30]:
print(result['result'])


Please select one of the following options to answer the question:

1) A foreign protein or particle that stimulates an immune response.
2) A substance that triggers the body to produce antibodies.
3) A specific type of histamine released by mast cells.
4) A chemical stored within cells that reacts with antigens.
5) A protein made by the body as a defense against foreign material.
6) Any substance that causes the body to launch an immune response.
